In [1]:
##从dgp生成数据
import os
import pyro
import torch
import pickle
import time

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import pyro.distributions as dist

from tqdm.auto import trange
from pyro.nn import PyroModule, PyroSample, PyroParam
from pyro.infer import MCMC, NUTS, SVI, Trace_ELBO, Predictive
from pyro.distributions import constraints
from pyro.infer.autoguide import AutoDiagonalNormal

from src.dgp_rff.deep_layer import DeepGP, DeepGPNoBias, DeepEnsembleGP
import os
import pickle
import numpy as np
from matplotlib import pyplot as plt

In [13]:
import torch
torch.cuda.is_available()
torch.cuda.current_device()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)
# cuda = torch.device('cuda') 

In [14]:
index = "dgp"
fold = "4_2"
#fold1: 2 layer x to y: 1 to 3 dim
#fold2: 3 layer x to y: 1 to 5 dim


In [15]:
class ModelDGP(PyroModule):
    np.random.seed(1)
    torch.manual_seed(1)
    def __init__(self, dim_list=[1,1,1], J_list=[50,10]):
        super().__init__()
        
        self.out_dim = dim_list[-1]
        self.model = DeepGP(dim_list, J_list)
        self.model.to('cuda')

    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        print("mushape",mu.shape)
        # batch shape | event shapt
        # 10000       |
        #scale = torch.FloatTensor([1e-8]*self.out_dim).cuda()
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise
        print("scale", scale.shape)

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample("obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)
            print("obs",obs.shape)


            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu


In [16]:
#fold 1:
# x_dim = 1
# y_dim = 3
# mymodel = ModelDGP(dim_list=[x_dim,10,y_dim], J_list=[50,10])
#fold 2:
x_dim = 1
y_dim = 4
#fold2:
#mymodel = ModelDGP(dim_list=[x_dim,10,5,y_dim], J_list=[50,50,20])
#fold3:
# mymodel = ModelDGP(dim_list=[x_dim,10,20,10,20,y_dim], J_list=[50,50,20,20,20])
#fold 4:
mymodel = ModelDGP(dim_list=[x_dim,10,5,10,y_dim], J_list=[50,20,20,20])


[SingleGP(
  (layers): PyroModuleList(
    (0): FirstLayer(
      (layer): PyroLinear(in_features=1, out_features=50, bias=False)
    )
    (1): SecondLayer(
      (layer): PyroLinear(in_features=100, out_features=10, bias=True)
    )
  )
), SingleGP(
  (layers): PyroModuleList(
    (0): FirstLayer(
      (layer): PyroLinear(in_features=10, out_features=20, bias=False)
    )
    (1): SecondLayer(
      (layer): PyroLinear(in_features=40, out_features=5, bias=True)
    )
  )
), SingleGP(
  (layers): PyroModuleList(
    (0): FirstLayer(
      (layer): PyroLinear(in_features=5, out_features=20, bias=False)
    )
    (1): SecondLayer(
      (layer): PyroLinear(in_features=40, out_features=10, bias=True)
    )
  )
), SingleGP(
  (layers): PyroModuleList(
    (0): FirstLayer(
      (layer): PyroLinear(in_features=10, out_features=20, bias=False)
    )
    (1): SecondLayer(
      (layer): PyroLinear(in_features=40, out_features=4, bias=True)
    )
  )
)]


In [17]:

# Set random seed for reproducibility
np.random.seed(1)

# Generate data
A =  np.array([[1,2,3]])

# number of training samples
n_train = 10000
x_obs = np.linspace(-0.2, 0.2, n_train)
noise = 0.2 * np.random.normal(0, 1, (n_train, y_dim))
print(noise.shape)
# tmp = x_obs.reshape(len(x_obs), 1) @ A
# The X and Y have to be at least 2-dim
xobs_torch = torch.from_numpy(x_obs).float().cuda().reshape(-1,1)
noise_torch = torch.from_numpy(noise).float().cuda()

y_obs = mymodel.forward(xobs_torch) + noise_torch
print("yobsshape",y_obs.shape)
# y_obs = np.concatenate((np.sin(tmp), np.cos(tmp)), axis=1) + noise

# Set plot limits and labels
xlims = [-0.2, 0.2]

x_train = x_obs
y_train = y_obs
print(y_train.shape)

# number of test samples
n_test = 100
x_test = np.linspace(xlims[0], xlims[1], n_test)
noise_test = 0.02 * np.random.normal(0, 1, (n_test, y_dim))
noise_test_torch = torch.from_numpy(noise_test).float().cuda()
print("noisetestshape",noise_test.shape,noise_test_torch.shape)
# tmp_test = x_test.reshape(len(x_test), 1) @ A
x_test_torch = torch.from_numpy(x_test).float().cuda().reshape(-1,1)
y_test = mymodel.forward(x_test_torch) + noise_test_torch

# y_test = np.concatenate((np.sin(tmp_test), np.cos(tmp_test)), axis=1) + noise_test

(10000, 4)
mushape torch.Size([10000, 4])
scale torch.Size([4])
obs torch.Size([10000, 4])
yobsshape torch.Size([10000, 4])
torch.Size([10000, 4])
noisetestshape (100, 4) torch.Size([100, 4])
mushape torch.Size([100, 4])
scale torch.Size([4])
obs torch.Size([100, 4])


In [7]:
os.chdir("../../")

In [18]:

cwd = os.getcwd()
print(cwd)


X_train_path = os.path.join(cwd, "folds", "synthetic_"+str(index)+"_fold_"+str(fold)+"_X_train.txt")
X_test_path = os.path.join(cwd, "folds", "synthetic_"+str(index)+"_fold_"+str(fold)+"_X_test.txt")
Y_train_path = os.path.join(cwd, "folds", "synthetic_"+str(index)+"_fold_"+str(fold)+"_Y_train.txt")
Y_test_path = os.path.join(cwd, "folds", "synthetic_"+str(index)+"_fold_"+str(fold)+"_Y_test.txt")

np.savetxt(X_train_path, x_train)
np.savetxt(X_test_path, x_test)
np.savetxt(Y_train_path, y_train.cpu().detach().numpy())
np.savetxt(Y_test_path, y_test.cpu().detach().numpy())

C:\Users\yuanq\OneDrive\Desktopold\SB\research\DGPII\program\DGP-RFF-main


In [8]:
np.random.seed(42)

# Read data
cwd = os.getcwd()
print(cwd)
index = "dgp"
fold = "4_1"

X_train_path = os.path.join(cwd, "folds", "synthetic_"+str(index)+"_fold_"+str(fold)+"_X_train.txt")
# X_test_path = os.path.join(cwd, "folds", "synthetic_"+str(index)+"_fold_"+str(fold)+"_X_test.txt")
Y_train_path = os.path.join(cwd, "folds", "synthetic_"+str(index)+"_fold_"+str(fold)+"_Y_train.txt")
# Y_test_path = os.path.join(cwd, "folds", "synthetic_"+str(index)+"_fold_"+str(fold)+"_Y_test.txt")

from sklearn.model_selection import train_test_split

# Load data
x_obs = np.loadtxt(X_train_path)  # Replace 'X_train_path' with your actual file path
y_obs = np.loadtxt(Y_train_path)  # Replace 'Y_train_path' with your actual file path

# Split data into training and test sets with 500 test points
x_train, x_test, y_train, y_test = train_test_split(x_obs, y_obs, test_size=500, random_state=42)

x_train = torch.from_numpy(x_train).float().reshape(-1,1)
y_train = torch.from_numpy(y_train).float()
x_test = torch.from_numpy(x_test).float().reshape(-1,1)
y_test = torch.from_numpy(y_test).float()

# # Check the sizes
# print(f'Training set size: {len(X_train)}')
# print(f'Test set size: {len(X_test)}')
# 
# x_obs = np.loadtxt(X_train_path)
# y_obs = np.loadtxt(Y_train_path)
# x_val = np.loadtxt(X_test_path)
# y_val = np.loadtxt(Y_test_path)

# Set plot limits and labels
xlims = [-0.2, 0.2]
# 